In [3]:
# ruff: noqa
import sys, os
sys.path.append(os.path.abspath("./../../feedback-grape"))
sys.path.append(os.path.abspath("./../"))
sys.path.append(os.path.abspath("./../../"))

# ruff: noqa
from feedback_grape.fgrape import optimize_pulse, evaluate_on_longer_time
from helpers import (
    init_grape_protocol,
    init_fgrape_protocol,
    test_implementations,
    generate_random_discrete_state,
    generate_random_bloch_state
)
from tqdm import tqdm
import jax
import jax.numpy as jnp
import numpy as np
from library.utils.FgResult_to_dict import FgResult_to_dict
import json

test_implementations()

jax.config.update("jax_enable_x64", True)

In [4]:
# Physical parameters
# (attention! #elements in density matrix grow as 4^N_chains)
N_chains = 3 # Number of parallel chains to simulate
gamma = 0.05 # Decay constant

# Training and evaluation parameters
training_params = {
    "N_samples": 10, # Number of random initial states to sample
    "N_training_iterations": 10000, # Number of training iterations
    "learning_rate": 0.001, # Learning rate
    "convergence_threshold": 1e-6,
    "batch_size": 8,
    "eval_batch_size": 16,
    "evaluation_time_steps": 100,
}
N_parallel_threads = 4 # Number of parallel threads for training
generate_state = generate_random_bloch_state

# Parameters to test

#num_time_steps : Number of time steps in the control pulse
#lut_depth : Depth of the lookup table for feedback
#reward_weights: Weights for the reward at each time step. Default only weights last timestep [0, 0, ... 0, 1]

experiments = [ # (timesteps, lut_depth, reward_weights, noise_level)
    (6,2,[1,1,1,1,1,1], 0.0),
    (6,2,[1,1,1,1,1,1], 0.5),
    (6,2,[1,1,1,1,1,1], 1.0),
    (6,2,[1,1,1,1,1,1], float('inf')),
    (7,2,[1,1,1,1,1,1,1], 0.0),
    (7,2,[1,1,1,1,1,1,1], 0.5),
    (7,2,[1,1,1,1,1,1,1], 1.0),
    (7,2,[1,1,1,1,1,1,1], float('inf')),
]

experiments_format = [ # Format of variables as tuples (name, type, required by grape, required by lut, required by rnn)
    ("t", int, True, True, True), # number timesteps (or segments)
    ("l", int, False, True, False), # LUT depth
    ("w", list, True, True, True), # reward weights
    ("noise", float, True, True, True), # noise level
]

# Check if experiment parameters are in specified format
for params in experiments:
    assert len(params) == len(experiments_format), "Experiment parameters length mismatch."
    for i, (param, (name, expected_type, req_grape, req_lut, req_rnn)) in enumerate(zip(params, experiments_format)):
        assert type(name) == str, "Parameter name must be a string."
        assert type(req_grape) == bool, "Requirement flags must be boolean."
        assert type(req_lut) == bool, "Requirement flags must be boolean."
        assert type(req_rnn) == bool, "Requirement flags must be boolean."
        assert type(param) == expected_type, f"Parameter '{name}' at index {i} must be of type {expected_type.__name__}."
        if type(param) == list:
            for j, item in enumerate(param):
                assert type(item) == int and 0 <= item <= 9, f"Item at index {j} in parameter '{name}' must be of type int and between 0 and 9."

def generate_param_paths(params, model_type):
    assert model_type in ["grape", "lut", "rnn"], "Invalid model type."

    parts = []
    for (name, expected_type, req_grape, req_lut, req_rnn), param in zip(experiments_format, params):
        required = (model_type == "grape" and req_grape) or (model_type == "lut" and req_lut) or (model_type == "rnn" and req_rnn)
        if required:
            if expected_type == list:
                param_str = "".join(str(x) for x in param)
            else:
                param_str = str(param)
            parts.append(f"{name}={param_str}")

    pstr = f"{model_type}_" + "_".join(parts)
    return (
        "./optimized_architectures/" + pstr + ".json",
        "./optimized_architectures/" + pstr + "_training_data.json",
        "./evaluation_results/" + pstr + ".npz"
    )

In [3]:
from concurrent.futures import ThreadPoolExecutor

def run_experiment(params):
    num_time_steps, lut_depth, reward_weights, noise_level = params
    print(f"Evaluating {params}")
    noisy_state = lambda key: generate_state(key, N_chains=N_chains, noise_level=noise_level)
    pure_state = lambda key: generate_state(key, N_chains=N_chains, noise_level=0.0)

    # Only test LUTs in this example
    path1, path2, path3 = generate_param_paths(params, "lut")
    if os.path.exists(path1):
        print(f"LUT already trained, skipping training.")
    else:
        print("Training LUT")

        best_result = None
        best_result_fidelities = None
        fidelities_each = []
        for s in tqdm(range(training_params["N_samples"])):
            system_params = init_fgrape_protocol(jax.random.PRNGKey(s), N_chains, gamma)

            # Train LUT
            result = optimize_pulse(
                U_0=noisy_state,
                C_target=pure_state,
                system_params=system_params,
                num_time_steps=num_time_steps,
                lut_depth=lut_depth,
                reward_weights=reward_weights,
                mode="lookup",
                goal="fidelity",
                max_iter=training_params["N_training_iterations"],
                convergence_threshold=training_params["convergence_threshold"],
                learning_rate=training_params["learning_rate"],
                evo_type="density",
                batch_size=training_params["batch_size"],
                eval_batch_size=1, # This evaluation is discarded
            )

            eval_result = evaluate_on_longer_time( # Evaluate on longer time and choose best LUT accordingly
                U_0 = pure_state,
                C_target = pure_state,
                system_params = system_params,
                optimized_trainable_parameters = result.optimized_trainable_parameters,
                num_time_steps = training_params["evaluation_time_steps"],
                evo_type = "density",
                goal = "fidelity",
                eval_batch_size = training_params["eval_batch_size"],
                mode = "lookup",
            )

            fidelities = eval_result.fidelity_each_timestep

            if best_result is None or np.mean(fidelities) > np.mean(best_result_fidelities):
                best_result = result
                best_result_fidelities = fidelities

            fidelities_each.append(float(np.mean(fidelities)))

        np.savez(path3, fidelities_lut=np.array(best_result_fidelities))

        with open(path1, "w") as f:
            json.dump(FgResult_to_dict(best_result), f)

        with open(path2, "w") as f:
            training_data = {
                "fidelities_each_sample": fidelities_each,
                "average_fidelity": float(np.mean(fidelities_each)),
                "training_params": training_params,
            }
            json.dump(training_data, f)

# Run experiments in parallel
if N_parallel_threads > 1:
    with ThreadPoolExecutor(max_workers=N_parallel_threads) as executor:
        executor.map(run_experiment, experiments)
else:
    for params in experiments:
        run_experiment(params)

# Play a sound when done
os.system('say "done."')

Evaluating (6, 2, [1, 1, 1, 1, 1, 1], 0.0)Evaluating (6, 2, [1, 1, 1, 1, 1, 1], 0.5)
Evaluating (6, 2, [1, 1, 1, 1, 1, 1], 1.0)
Training LUT
Training LUT

Training LUT
Evaluating (6, 2, [1, 1, 1, 1, 1, 1], inf)
Training LUT


  0%|          | 0/10 [00:00<?, ?it/s]


 10%|█         | 1/10 [05:07<46:11, 307.96s/it]


 20%|██        | 2/10 [09:59<39:45, 298.25s/it]


 30%|███       | 3/10 [16:03<38:19, 328.47s/it]


 40%|████      | 4/10 [21:43<33:18, 333.05s/it]


 50%|█████     | 5/10 [27:18<27:47, 333.53s/it]


 60%|██████    | 6/10 [33:47<23:29, 352.37s/it]


 70%|███████   | 7/10 [40:04<18:01, 360.52s/it]


2025-10-30 16:28:27.924667: E external/xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
 80%|████████  | 8/10 [48:08<13:19, 399.87s/it]2025-10-30 16:33:23.046511: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 6m55.129933s

********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/

Evaluating (7, 2, [1, 1, 1, 1, 1, 1, 1], 0.0)
Training LUT


100%|██████████| 10/10 [1:07:39<00:00, 405.92s/it]


Evaluating (7, 2, [1, 1, 1, 1, 1, 1, 1], 0.5)
Training LUT


...

Evaluating (7, 2, [1, 1, 1, 1, 1, 1, 1], 1.0)
Training LUT





100%|██████████| 10/10 [1:10:38<00:00, 423.88s/it]


Evaluating (7, 2, [1, 1, 1, 1, 1, 1, 1], inf)
Training LUT




 10%|█         | 1/10 [10:21<1:33:17, 621.90s/it]


2025-10-30 17:07:10.770266: E external/xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-10-30 17:08:49.472204: E external/xla/xla/service/slow_operation_alarm.cc:140] The operation took 3m38.702445s

********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-10-30 17:09:42.436775: E external/xla/xla/service/slow_operation_alarm.cc:73] 
********************************
[Compiling module jit_step] Very slow compile? If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2025-10-30 17:10:26.06

0